In [1]:

import DataSetQuery
import AsyncDataSetQuery
import datetime
import json
from pandas.io.json import json_normalize
import pandas as pd
import datetime
from datetime import timedelta
from dateutil.relativedelta import *

import MalardHelpers as m

environmentName = 'DEVv2'
#Sets the URL of the Malard ServiceGateway.
query = DataSetQuery.DataSetQuery('http://localhost:9000',environmentName)

asyncquery = AsyncDataSetQuery.AsyncDataSetQuery( 'ws://localhost:9000',environmentName)


In [2]:
query.getParentDataSets()

'[{"name":"cryotempo"},{"name":"jatest"},{"name":"mtngla"},{"name":"mtnglauat"}]'

In [3]:
query.getDataSets('mtngla')

'[{"name":"tdx","region":"himalayas"},{"name":"tdxtest1","region":"himalayas"},{"name":"srtm","region":"himalayas"},{"name":"testdataset4","region":"himalayas"},{"name":"ADwithTDX","region":"alaska"},{"name":"Ready1","region":"himalayas"},{"name":"publishtest","region":"himalayas"}]'

In [4]:
parentDs = 'mtnglauat'
dataSet = 'tdx'
region = 'himalayas'

bbox = json.loads(query.getDataSetBoundingBox( parentDs,dataSet,region ))

In [5]:
print(bbox)

{'gridCellMinX': -1900000, 'gridCellMaxX': 1700000, 'gridCellMinY': -1300000, 'gridCellMaxY': 900000, 'minTime': 1279266673, 'maxTime': 1556381631, 'totalPoints': 5503342642, 'numberOfShards': 55681}


In [6]:
minX = bbox['gridCellMinX']
maxX = bbox['gridCellMaxX']
minY = bbox['gridCellMinY']
maxY = bbox['gridCellMaxY']
minT = datetime.datetime.fromtimestamp(bbox['minTime'])
maxT = datetime.datetime.fromtimestamp(bbox['maxTime'])

gridCellsDf = json_normalize(json.loads(query.getGridCells( parentDs, dataSet, region, minX, maxX, minY, maxY, minT, maxT )))


In [7]:
shardsDf = json_normalize(json.loads(query.getShards( parentDs, dataSet, region, minX, maxX, minY, maxY, minT, maxT )))

print(shardsDf.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55681 entries, 0 to 55680
Data columns (total 8 columns):
maxT              55681 non-null int64
maxX              55681 non-null float64
maxY              55681 non-null float64
minT              55681 non-null int64
minX              55681 non-null float64
minY              55681 non-null float64
numberOfPoints    55681 non-null int64
shardName         55681 non-null object
dtypes: float64(4), int64(3), object(1)
memory usage: 3.4+ MB
None


In [8]:


def checkGridCellByMonth( gc ):
    count = 0 
    gcStartTime = datetime.datetime.fromtimestamp(gc['minTime'])
    gcEndTime = datetime.datetime.fromtimestamp(gc['maxTime'])
    minX = int(gc['gridCellMinX'])
    maxX = int(gc['gridCellMaxX'])
    minY = int(gc['gridCellMinY'])
    maxY = int(gc['gridCellMaxY'])
    
    adjStartTime = datetime.datetime( gcStartTime.year, gcStartTime.month, 1, 0, 0,0  )  
    incStartTime = adjStartTime +relativedelta(months=+1)-timedelta(seconds=1)
    
    while adjStartTime < gcEndTime:
        minT = adjStartTime
        maxT = incStartTime
        
        cacheResult = asyncquery.executeQuery( parentDs, dataSet, region, minX, maxX, minY, maxY, minT, maxT,[],[] )
        
        if cacheResult.status != 'Error: Empty resultset.':
            df = m.getDataFrameFromNetCDF(cacheResult.resultFileName)
            count = count + len(df['x'])
            query.releaseCache(cacheResult.resultFileName)
        
        adjStartTime = incStartTime
        incStartTime = incStartTime +relativedelta(months=+1)-timedelta(seconds=1)
    
    return count

def checkGridCell( gc ):
    count = 0 
    gcStartTime = datetime.datetime.fromtimestamp(gc['minTime'])
    gcEndTime = datetime.datetime.fromtimestamp(gc['maxTime'])
    minX = int(gc['gridCellMinX'])
    maxX = int(gc['gridCellMaxX'])
    minY = int(gc['gridCellMinY'])
    maxY = int(gc['gridCellMaxY'])
    minT = gcStartTime
    maxT = gcEndTime
        
    cacheResult = asyncquery.executeQuery( parentDs, dataSet, region, minX, maxX, minY, maxY, minT, maxT,['x'],[] )
        
    if cacheResult.status != 'Error: Empty resultset.':
        df = m.getDataFrameFromNetCDF(cacheResult.resultFileName)
        count = count + len(df['x'])
        query.releaseCache(cacheResult.resultFileName)
    else:
        print("Unexpected error... ")
        
    return count

In [9]:
totalcount = 0

for rowId, gridCell  in gridCellsDf.iterrows():
    count = checkGridCell(gridCell)
    totalcount = count + totalcount
    print("%d CumCount=%d MinX=%d MinY=%d QueryCount=%d GridCellCount=%d"%(rowId, totalcount, gridCell['gridCellMinX'], gridCell['gridCellMinY'],count, gridCell['totalPoints']))


0 CumCount=8 MinX=1500000 MinY=-400000 QueryCount=8 GridCellCount=8
1 CumCount=21 MinX=200000 MinY=800000 QueryCount=13 GridCellCount=13
2 CumCount=728 MinX=1500000 MinY=-1200000 QueryCount=707 GridCellCount=707
3 CumCount=748 MinX=1600000 MinY=-800000 QueryCount=20 GridCellCount=20
4 CumCount=1272 MinX=1600000 MinY=-1000000 QueryCount=524 GridCellCount=524
5 CumCount=1678 MinX=1100000 MinY=500000 QueryCount=406 GridCellCount=406
6 CumCount=3024 MinX=-1700000 MinY=-900000 QueryCount=1346 GridCellCount=1346
7 CumCount=9748 MinX=1000000 MinY=-1100000 QueryCount=6724 GridCellCount=6724
8 CumCount=13103 MinX=500000 MinY=800000 QueryCount=3355 GridCellCount=3355
9 CumCount=13120 MinX=-1700000 MinY=400000 QueryCount=17 GridCellCount=17
10 CumCount=29931 MinX=400000 MinY=500000 QueryCount=16811 GridCellCount=16811
11 CumCount=98870 MinX=500000 MinY=700000 QueryCount=68939 GridCellCount=68939
12 CumCount=103713 MinX=1200000 MinY=-1100000 QueryCount=4843 GridCellCount=4843
13 CumCount=113593 Mi

KeyboardInterrupt: 

In [10]:
#MinX=-600000 MinY=300000 QueryCount=64074 GridCellCount=64077

MinX=-600000
MaxX = MinX + 100000
MinY=300000
MaxY=MinY+100000

shards = json_normalize(json.loads(query.getShards(parentDs, dataSet, region, MinX, MaxX, MinY, MaxY, minT, maxT )))


In [11]:
count = 0

for s in shards['shardName']:
    df = m.getDataFrameFromNetCDF(s)
    time = datetime.datetime.fromtimestamp(df['time'].min())
    startTime = datetime.datetime( time.year, time.month, 1, 0, 0,0  )  
    endTime = startTime +relativedelta(months=+1)-timedelta(seconds=1)
    
    result = asyncquery.executeQuery( parentDs, dataSet, region, MinX, MaxX, MinY, MaxY, startTime, endTime,[],[]   )
    dfRes = m.getDataFrameFromNetCDF( result.resultFileName )
    
    print( 'StartTime %s EndTime %s ShardCount %d QueryCount %d' % (startTime, endTime, len(df['x']), len(dfRes['x'])))
    
    count = count + len(df['x'])
    
print(count)

StartTime 2016-06-01 00:00:00 EndTime 2016-06-30 23:59:59 ShardCount 2404 QueryCount 2404
StartTime 2016-04-01 00:00:00 EndTime 2016-04-30 23:59:59 ShardCount 5086 QueryCount 5086
StartTime 2017-03-01 00:00:00 EndTime 2017-03-31 23:59:59 ShardCount 17407 QueryCount 17407
StartTime 2016-12-01 00:00:00 EndTime 2016-12-31 23:59:59 ShardCount 4315 QueryCount 4315
StartTime 2018-06-01 00:00:00 EndTime 2018-06-30 23:59:59 ShardCount 552 QueryCount 552
StartTime 2016-09-01 00:00:00 EndTime 2016-09-30 23:59:59 ShardCount 1641 QueryCount 1641
StartTime 2017-02-01 00:00:00 EndTime 2017-02-28 23:59:59 ShardCount 7248 QueryCount 7248
StartTime 2016-11-01 00:00:00 EndTime 2016-11-30 23:59:59 ShardCount 7541 QueryCount 7541
StartTime 2017-01-01 00:00:00 EndTime 2017-01-31 23:59:59 ShardCount 16407 QueryCount 16407
StartTime 2016-10-01 00:00:00 EndTime 2016-10-31 23:59:59 ShardCount 111 QueryCount 111
StartTime 2016-07-01 00:00:00 EndTime 2016-07-31 23:59:59 ShardCount 1314 QueryCount 1314
StartTime 

In [ ]:
maxT = datetime.datetime(2019,12,30,23,59,59)
minT = datetime.datetime(2012,1,1,0,0,0)

result = asyncquery.executeQuery( parentDs, dataSet, region, MinX, MaxX, MinY, MaxY, minT, maxT,[],[]   )

dfRes = m.getDataFrameFromNetCDF( result.resultFileName )

print(len(dfRes['x'])  ) 

dfRes.to_csv("first.csv")